Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

/home/fja0568/src/brownbear
/home/fja0568/src/brownbear/universe/dow30-galaxy


In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# remove the exchange from the beginning of the symbol
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()
dow30['Symbol'] = dow30.apply(_symbol, axis=1)

dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting(as of September 1, 2020'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Pharmaceutical industry
AAPL,Apple Inc.,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar Inc.,Construction and Mining
CVX,Chevron Corporation,Petroleum industry
CSCO,Cisco Systems,Information technology
KO,The Coca-Cola Company,Food industry


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and Mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services'}

In [7]:
# map dow30 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
dow30.index = dow30.index.str.replace('.', '-')
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AAPL,Apple Inc.,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar Inc.,US Stocks:Industrials
CVX,Chevron Corporation,US Stocks:Energy
CSCO,Cisco Systems,US Stocks:Technology
KO,The Coca-Cola Company,US Stocks:Consumer Staples


In [8]:
# make symbols list
symbols = list(dow30.index)
#symbols

In [9]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..............................


In [10]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,...,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT,DIS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,138.20,84.53,136.29,24.70,112.79,75.25,86.83,22.96,34.62,nan,...,43.85,75.38,59.24,92.10,91.65,35.86,63.48,65.25,74.44,87.38
2015-01-05,135.08,82.29,134.67,24.00,112.00,71.28,83.36,22.51,34.62,nan,...,43.14,75.02,58.17,90.99,90.14,35.56,62.07,63.96,74.22,86.10
2015-01-06,133.64,80.54,130.33,24.00,110.69,70.82,83.32,22.50,34.88,nan,...,42.89,74.68,57.20,90.17,89.96,35.92,61.67,64.13,74.79,85.64
2015-01-07,134.61,82.30,134.88,24.34,112.40,71.92,83.25,22.71,35.32,nan,...,43.77,75.07,56.93,91.71,90.88,35.68,62.50,65.77,76.78,86.52
2015-01-08,137.84,83.47,134.40,25.27,114.39,72.66,85.16,22.88,35.74,nan,...,44.79,75.93,58.59,93.61,95.22,36.45,63.34,66.58,78.40,87.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-30,172.73,118.59,222.04,119.05,210.71,173.59,87.18,43.02,51.60,53.01,...,134.43,138.87,245.80,129.65,335.13,60.41,210.35,38.01,152.79,148.01
2020-12-01,170.52,119.93,225.08,122.72,213.01,173.26,87.45,43.54,52.04,53.53,...,135.17,139.37,241.35,133.41,339.97,60.58,211.20,38.46,152.64,149.44
2020-12-02,171.85,122.33,226.56,123.08,223.85,173.87,89.87,43.89,52.11,53.20,...,135.31,138.36,220.78,134.04,346.31,61.36,210.18,39.84,150.52,153.61


In [11]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['MMM']

8.82862336113217

In [12]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [13]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['MMM']

0.20913230374939587

In [14]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['MMM']

0.10253705304570353

In [15]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['MMM'])

Date
2015-01-31   136.72
2015-02-28   142.95
2015-03-31   139.82
2015-04-30   132.56
2015-05-31   135.69
2015-06-30   131.62
2015-07-31   129.09
2015-08-31   122.09
2015-09-30   121.77
2015-10-31   135.03
2015-11-30   135.37
2015-12-31   130.24
2016-01-31   130.55
2016-02-29   136.60
2016-03-31   145.10
2016-04-30   145.76
2016-05-31   147.55
2016-06-30   153.51
2016-07-31   156.35
2016-08-31   158.10
2016-09-30   155.45
2016-10-31   145.81
2016-11-30   152.46
2016-12-31   158.52
2017-01-31   155.19
2017-02-28   166.51
2017-03-31   170.95
2017-04-30   174.98
2017-05-31   183.79
2017-06-30   187.13
2017-07-31   180.82
2017-08-31   184.71
2017-09-30   189.75
2017-10-31   208.09
2017-11-30   220.91
2017-12-31   213.85
2018-01-31   227.60
2018-02-28   215.24
2018-03-31   200.63
2018-04-30   177.66
2018-05-31   181.49
2018-06-30   181.02
2018-07-31   195.37
2018-08-31   195.39
2018-09-30   195.19
2018-10-31   176.25
2018-11-30   193.92
2018-12-31   177.71
2019-01-31   186.81
2019-02-28   19

In [16]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['MMM']

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2020-08-31    0.09
2020-09-30   -0.02
2020-10-31   -0.00
2020-11-30    0.09
2020-12-31   -0.00
Freq: M, Name: MMM, Length: 72, dtype: float64

In [17]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['MMM']

0.23065284848148046

In [18]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [19]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [20]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')